In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("/home/oscb/Mechanistic-Interpretation-Material-Inference/mimi/data/tr_data_type.csv")

In [4]:
df.columns

Index(['id', 'premise1', 'premise2', 'conclusion', 'type', 'new_type',
       'type_amr_op', 'Remarks (Ian)  '],
      dtype='object')

In [6]:
df[df["new_type"] == "substitution"] 

,id,premise1,premise2,conclusion,type,new_type,type_amr_op,Remarks (Ian)
0,0,climate means the average temperature / weathe...,winter has cold temperatures / weather,winter can cause the climate in an area to bec...,init,substitution,arg_substitution: (temperature/weather -> clim...,"invalid, but by equalizing similar notions, ye..."
8,8,the formation of diamonds requires intense pre...,the pressure is intense deep below earth 's crust,the formation of diamonds occurs deep below th...,init,substitution,frame_substitution: (requires -> form; intens...,Not sure why this isn't arg_substitution. But ...
11,11,mosquitos eat their food source,mosquitos eat blood of animals,animals are a source of food for mosquitos,init,substitution,frame_substitution: (animals -> food source),Why isn't this arg_substitution?
18,18,pollution has a negative impact on the environ...,runoff from a cropland causes water pollution,runoff from cropland has a negative impact on ...,init,substitution,arg_substitution: (pollution -> runoff from a ...,NaN
25,25,the nucleus of an atom is positve in electric ...,the nucleus is the center of an atom,the positive charge is concentrated in the cen...,init,substitution,frame_insertion,NaN
...,...,...,...,...,...,...,...,...
4821,4821,an example of a physical change is squeezing s...,"during digestion, food is squeezed through the...",an example of a physical change is squeezing f...,init,substitution,NaN,NaN
4853,4853,blowing air into a whistle causes air to vibra...,vibrations are a source of sound,the air vibrations inside the whistle are a so...,init,substitution,NaN,NaN
4993,4993,dogs will inherit inherited characteristics fr...,the color of / coloration of fur is an inherit...,dogs will have the same coloration of fur as t...,init,substitution,NaN,NaN
4997,4997,an example of camouflage is organism having th...,some environments change color,an example of camouflage is organism changing ...,init,substitution,NaN,NaN


In [1]:
data_folder= "/home/oscb/Mechanistic-Interpretation-Material-Inference/mimi/data/examples_100"

In [2]:
from pathlib import Path

folder = Path(data_folder)

for file_path in folder.iterdir():          # yields Path objects for *everything* inside
    if file_path.is_file():                 # skip sub-dirs if you only want files
        print(file_path.name) 

if_then_100.csv
cond_frame_100.csv
arg_ins_100.csv
frame_sub_100.csv
arg_substitution_100.csv
frame_conj_100.csv
arg_pred_gen_100.csv
example_100.csv
pred_sub_100.csv
property_inheritance_100.csv


In [3]:
from pathlib import Path
import pandas as pd

folder = Path(data_folder)        # ⇦ put your folder here

for csv_path in folder.glob("*.csv"):   # ── outer loop: each CSV file
    print(f"↳ processing {csv_path.name}")

    df = pd.read_csv(csv_path)          # read the file into a DataFrame

    for row in df.itertuples(index=False):  # ── inner loop: each row (fastest)
        # row is a lightweight named-tuple; access by attribute or position
        # e.g.  row.ColumnA   or   row[0]
        print(row)
        
        # example: accumulate a value
        # total += row.amount


↳ processing if_then_100.csv
Pandas(P1='solar panels requires sunlight to generate electricity', P2='heavy cloud cover blocks sunlight', C='if there is heavy cloud cover, then the solar panels cannot generate electricity')
Pandas(P1='car engines requires fuel to run', P2='an empty gas tank blocks fuel', C='if there is an empty gas tank, then the car engines cannot run')
Pandas(P1='campfires requires dry wood to burn', P2='recent rain blocks dry wood', C='if there is recent rain, then the campfires cannot burn')
Pandas(P1='wind turbines requires wind to produce power', P2='a calm day blocks wind', C='if there is a calm day, then the wind turbines cannot produce power')
Pandas(P1='humans requires oxygen to breathe properly', P2='thin atmospheric air blocks oxygen', C='if there is thin atmospheric air, then the humans cannot breathe properly')
Pandas(P1='submarines requires compressed air to surface safely', P2='air‑system failure blocks compressed air', C='if there is air‑system failure,

In [4]:
from pathlib import Path
import pandas as pd
import re
from itertools import groupby

# ── tweak this ──────────────────────────────────────────────────────────────────
FOLDER = Path(data_folder)          # <-- put the directory that holds your .csv files
# ────────────────────────────────────────────────────────────────────────────────

def derive_regex_pattern(strings):
    """
    Find the common skeleton of a list of very similar sentences and turn it into
    a regex pattern with '(.+?)' capture slots where they differ.

    Returns (pattern_str, compiled_regex).
    """
    # tokenise: words OR punctuation
    tokenised = [re.findall(r"\w+|[^\w\s]", s) for s in strings]
    base = tokenised[0]

    pattern_tokens = []
    for i, tok in enumerate(base):
        # keep the token only if every sentence has *that same token* in the same spot
        if all(i < len(ts) and ts[i] == tok for ts in tokenised[1:]):
            pattern_tokens.append(re.escape(tok))
        else:
            pattern_tokens.append(r"(.+?)")          # slot for the variable part

    # collapse consecutive slots into a single '(.+?)'
    pattern_tokens = [
        t if t != r"(.+?)" else r"(.+?)"
        for t, _ in groupby(pattern_tokens)
    ]

    pattern = r"\s*".join(pattern_tokens)            # allow flexible spacing
    return pattern, re.compile(rf"^{pattern}$", re.I)

def capture_variables(regex, sentence):
    """Return the tuple of variable pieces for one sentence (or () if no match)."""
    m = regex.match(sentence)
    return m.groups() if m else ()

# ── main loop over every CSV ────────────────────────────────────────────────────
for csv_path in FOLDER.glob("*.csv"):
    df = pd.read_csv(csv_path)
    print(f"\n=== {csv_path.name} ===")

    for col in df.columns:               # do P1, P2, C … whatever the headers are
        sentences = df[col].dropna().astype(str).tolist()
        pattern_str, pattern_re = derive_regex_pattern(sentences)

        print(f"\n{col} pattern:")
        print(f"  {pattern_str}")

        print("  variable parts per row:")
        for idx, sentence in enumerate(sentences, 1):
            print(f"    row {idx:>2}: {capture_variables(pattern_re, sentence)}")



=== if_then_100.csv ===

P1 pattern:
  (.+?)
  variable parts per row:
    row  1: ('solar panels requires sunlight to generate electricity',)
    row  2: ('car engines requires fuel to run',)
    row  3: ('campfires requires dry wood to burn',)
    row  4: ('wind turbines requires wind to produce power',)
    row  5: ('humans requires oxygen to breathe properly',)
    row  6: ('submarines requires compressed air to surface safely',)
    row  7: ('plants requires water to grow',)
    row  8: ('smartphones requires battery charge to operate',)
    row  9: ('aquariums requires filtered water to support fish',)
    row 10: ('photographs requires light to form images',)
    row 11: ('ice skaters requires ice to glide',)
    row 12: ('farm crops requires fertile soil to produce yield',)
    row 13: ('microphones requires electrical power to amplify sound',)
    row 14: ('airplanes requires lift to stay airborne',)
    row 15: ('fire extinguishers requires pressurized agent to function',)
 

In [5]:
from pathlib import Path
import pandas as pd
import re
from itertools import groupby

###############################################################################
# CONFIG –– change ONLY the folder path if your CSVs live elsewhere
###############################################################################
CSV_DIR = Path(data_folder)                   # directory holding the .csv files
TARGET_COLS = ("P1", "P2", "C")          # columns to analyse (case-sensitive)
###############################################################################

def build_template(sentences):
    """
    Return a regex template and compiled pattern that capture ONLY the words
    that differ across `sentences` (which should all be very similar English
    clauses).

    The template will contain the minimal number of (.+?) slots required.
    """
    # ❶ Tokenise into "word OR punctuation"
    tokenised = [re.findall(r"\w+|[^\w\s]", s) for s in sentences]
    base = tokenised[0]                        # use the first row as reference

    # ❷ Decide, token-by-token, which positions are constant
    tokens = []
    for i, tok in enumerate(base):
        constant = all(i < len(ts) and ts[i] == tok for ts in tokenised[1:])
        tokens.append(re.escape(tok) if constant else r"(.+?)")

    # ❸ Collapse consecutive capture slots → a single '(.+?)'
    tokens = [t if t != r"(.+?)" else r"(.+?)" for t, _ in groupby(tokens)]

    # ❹ Join back with \s* to allow flexible spacing
    template = r"\s*".join(tokens)
    return template, re.compile(rf"^{template}$", flags=re.I)

def analyse_file(path):
    df = pd.read_csv(path)
    print(f"\n================ {path.name} ================")

    for col in TARGET_COLS:
        if col not in df.columns:
            continue                               # skip missing columns

        lines = df[col].dropna().astype(str).tolist()
        template, pattern = build_template(lines)

        # –– display ––
        print(f"\n{col} TEMPLATE:\n  {template}")

        print("VARIABLE CAPTURES:")
        for i, line in enumerate(lines, 1):
            caps = pattern.match(line).groups()
            print(f"  row {i:>2}: {caps}")

# ── run over every CSV ────────────────────────────────────────────────────────
for csv_file in sorted(CSV_DIR.glob("*.csv")):
    analyse_file(csv_file)



================ arg_ins_100.csv ================

P1 TEMPLATE:
  (.+?)
VARIABLE CAPTURES:
  row  1: ('solar energy comes from the sun',)
  row  2: ('rainwater falls from clouds',)
  row  3: ('table salt contains sodium chloride',)
  row  4: ('espresso contains high caffeine',)
  row  5: ('lava emerges from volcanoes',)
  row  6: ('fog forms near cool ground',)
  row  7: ('honey is produced by bees',)
  row  8: ('amber forms from tree resin',)
  row  9: ('stainless steel contains chromium',)
  row 10: ('yogurt results from milk fermentation',)
  row 11: ('dew collects on surfaces overnight',)
  row 12: ('lightning is caused by electric discharge',)
  row 13: ('pollen is carried by wind',)
  row 14: ('iceberg breaks off glaciers',)
  row 15: ('sea breeze blows from ocean to land',)
  row 16: ('peat accumulates in bogs',)
  row 17: ('silt is deposited by rivers',)
  row 18: ('glacier meltwater flows from melting ice',)
  row 19: ('quartz sand is composed of silica',)
  row 20: ('propoli

In [15]:
import re
import pandas as pd
import csv
# ---------- helpers ----------
# phrase-level substitutions you showed in your example
PHRASE_REPLACEMENTS = [
    (' is a kind of ',  ', is a kind of, '),
    (' that ',           ', that, '),
    (' comes from ',     ', comes, from '),
    (' falls from ',     ', falls, from '),
    (' flows from ',     ', flows, from '),
    (' emerges from ',   ', emerges, from '),
    (' forms from ',     ', forms, from '),
    (' blows from ',     ', blows, from '),
    (' precipitates from ', ', precipitates, from '),
    (' crystallizes from ', ', crystallizes, from '),
    (' erupts from ',    ', erupts, from '),
    (' results from ',   ', results, from '),
    (' rises from ',     ', rises, from '),
    (' contains ',       ', contains, '),          # “table salt, contains, sodium chloride”
    (' hosts ',          ', hosts, '),
    (' ages in ',        ', ages, in '),
    (' is caused by ',   ', is caused, by '),
    (' is produced by ', ', is produced, by '),
    (' is collected by ',', is collected, by '),
    (' is secreted by ', ', is secreted, by '),
    (' is made from ',   ', is made, from '),
    (' is distilled from ', ', is distilled, from '),
    (' is extracted from ', ', is extracted, from '),
    (' is pressed from ', ', is pressed, from '),
    (' is ground from ', ', is ground, from '),
    (' is derived from ', ', is derived, from '),
    (' is fermented from ', ', is fermented, from '),
    (' is brewed from ', ', is brewed, from '),
    (' is milled from ', ', is milled, from '),
    (' is processed from ', ', is processed, from '),
]

# fallback pattern: any “verb PREP” we haven’t listed above
PREP_WORDS = r'(from|by|in|on|over|with|into|off)'
FALLBACK = re.compile(r'\b([a-z]+(?:s|es|ed|ing)?)\s' + PREP_WORDS + r'\b')

def transform_sentence(sentence: str) -> str:
    s = ' ' + sentence.strip() + ' '          # pad to make the simple replacements work
    for old, new in PHRASE_REPLACEMENTS:
        s = s.replace(old, new)

    # generic split for unseen “verb prep” pairs
    s = FALLBACK.sub(r', \1, \2', s)

    # clean up whitespace
    s = re.sub(r'\s+', ' ', s).strip()
    s = re.sub(r',\s+', ', ', s)       # exactly one space after every comma
    return s

# ---------- I/O ----------
df = pd.read_csv('/home/oscb/Mechanistic-Interpretation-Material-Inference/mimi/data/examples_100/arg_ins_100.csv')              # your pasted dataset
for col in df.columns:
    df[col] = df[col].apply(transform_sentence)

df.to_csv(
    'transformed.csv',
    index=False,
    quoting=csv.QUOTE_NONE,   # <-- no double-quotes
    escapechar='\\',          #   (only used if a stray " appears)
)
print('Done!  ➜  transformed.csv')


Done!  ➜  transformed.csv


In [17]:
import re
import csv
import pandas as pd

# ---------------- helpers ----------------
REPLACEMENTS = [
    (' is a kind of ',  ', is a kind of, '),
    (' that ',           ', that, '),
    (' comes from ',     ', comes, from '),
    (' falls from ',     ', falls, from '),
    (' flows from ',     ', flows, from '),
    (' emerges from ',   ', emerges, from '),
    (' forms from ',     ', forms, from '),
    (' blows from ',     ', blows, from '),
    (' precipitates from ', ', precipitates, from '),
    (' crystallizes from ', ', crystallizes, from '),
    (' erupts from ',    ', erupts, from '),
    (' results from ',   ', results, from '),
    (' rises from ',     ', rises, from '),
    (' contains ',       ', contains, '),
    (' hosts ',          ', hosts, '),
    (' ages in ',        ', ages, in '),
    (' is caused by ',   ', is caused, by '),
    (' is produced by ', ', is produced, by '),
    (' is collected by ',', is collected, by '),
    (' is secreted by ', ', is secreted, by '),
    (' is made from ',   ', is made, from '),
    (' is distilled from ', ', is distilled, from '),
    (' is extracted from ', ', is extracted, from '),
    (' is pressed from ', ', is pressed, from '),
    (' is ground from ', ', is ground, from '),
    (' is derived from ', ', is derived, from '),
    (' is fermented from ', ', is fermented, from '),
    (' is brewed from ', ', is brewed, from '),
    (' is milled from ', ', is milled, from '),
    (' is processed from ', ', is processed, from ')
]

PREP_FALLBACK = re.compile(
    r'\b([a-z]+(?:s|es|ed|ing)?)\s(from|by|in|on|over|with|into|off)\b'
)

def phrase_split(sentence: str) -> list[str]:
    """Return a list of comma-separated units for one sentence."""
    s = ' ' + sentence.strip() + ' '
    for old, new in REPLACEMENTS:
        s = s.replace(old, new)
    s = PREP_FALLBACK.sub(r', \1, \2', s)
    parts = [p.strip() for p in s.split(',') if p.strip()]
    return parts

# ---------------- transform ----------------
df_in = pd.read_csv('/home/oscb/Mechanistic-Interpretation-Material-Inference/mimi/data/examples_100/arg_ins_100.csv')          # P1, P2, P3
rows_out = []

for _, row in df_in.iterrows():
    pieces = []
    for col in df_in.columns:
        pieces.extend(phrase_split(row[col]))
    rows_out.append(pieces)

# rectangle: pad shorter rows so every row has the same width
max_len = max(len(r) for r in rows_out)
rows_out = [r + ['']*(max_len-len(r)) for r in rows_out]

# build new DataFrame: C1, C2, C3, ...
col_names = [f'C{i}' for i in range(1, max_len+1)]
df_out = pd.DataFrame(rows_out, columns=col_names)

# ---------------- save ----------------
df_out.to_csv(
    'transformed.csv',
    index=False,
    quoting=csv.QUOTE_NONE,   # <- NO surrounding quotes
    escapechar='\\',          #   (only used if a stray " appears)
)

print(f'Done!  {len(df_out)} rows × {len(df_out.columns)} columns  ➜  transformed.csv')


Done!  100 rows × 13 columns  ➜  transformed.csv


In [ ]:
# ──────────────────────────────────────────────────────────────────────────────
# phrase_widen.py
#   usage:
#       python phrase_widen.py  original.csv   transformed.csv
# ──────────────────────────────────────────────────────────────────────────────

import re, csv, sys, pandas as pd

# --- phrase-level substitutions you asked for --------------------------------
REPLACEMENTS = [
    (' is a kind of ',  ', is a kind of, '),
    (' that ',           ', that, '),
    (' comes from ',     ', comes, from '),
    (' falls from ',     ', falls, from '),
    (' flows from ',     ', flows, from '),
    (' emerges from ',   ', emerges, from '),
    (' forms from ',     ', forms, from '),
    (' blows from ',     ', blows, from '),
    (' precipitates from ', ', precipitates, from '),
    (' crystallizes from ', ', crystallizes, from '),
    (' erupts from ',    ', erupts, from '),
    (' results from ',   ', results, from '),
    (' rises from ',     ', rises, from '),
    (' contains ',       ', contains, '),
    (' hosts ',          ', hosts, '),
    (' ages in ',        ', ages, in '),
    (' is caused by ',   ', is caused, by '),
    (' is produced by ', ', is produced, by '),
    (' is collected by ',', is collected, by '),
    (' is secreted by ', ', is secreted, by '),
    (' is made from ',   ', is made, from '),
    (' is distilled from ', ', is distilled, from '),
    (' is extracted from ', ', is extracted, from '),
    (' is pressed from ', ', is pressed, from '),
    (' is ground from ', ', is ground, from '),
    (' is derived from ', ', is derived, from '),
    (' is fermented from ', ', is fermented, from '),
    (' is brewed from ', ', is brewed, from '),
    (' is milled from ', ', is milled, from '),
    (' is processed from ', ', is processed, from ')
]

# catch-all: verb + prep we haven't listed
PREP_FALLBACK = re.compile(
    r'\b([a-z]+(?:s|es|ed|ing)?)\s(from|by|in|on|over|with|into|off)\b'
)

import re

# 1️⃣  unchanged “long phrases” first – keep exactly as before
LONG_PHRASE_REPL = [
    (' is a kind of ', ', is a kind of, '),
    (' results from ', ', results, from '),
    (' crystallizes from ', ', crystallizes, from '),
    (' denotes ',          ', denotes, '),
    (' means ',            ', means, '),
    (' implies ',          ', implies, '),
    (' signifies ',        ', signifies, '),
    (' describes ',        ', describes, '),
    (' refers to ',        ', refers, to '),      # two-word verb phrase
    (' indicates ',        ', indicates, '),
    (' entails ',          ', entails, '),
    (' represents ',       ', represents, '),
]

# 2️⃣  new REGEX-based splitter to cover the shorter verbs / patterns
IS_A_RX   = re.compile(r'\sis a\s(?!kind of\b)', flags=re.I)
IS_RX     = re.compile(r'\sis\s(?!a\b|a kind of\b)', flags=re.I)
HAS_RX    = re.compile(r'\shas\s', flags=re.I)

# single-word verbs that appear in the second file but weren’t in the first
VERBS = [
    'conducts', 'orbits', 'produces', 'transports', 'provides', 'forms',
    'releases', 'uses', 'quantifies', 'includes', 'emits', 'detects', 'kills',
    'stores', 'powers', 'rotates', 'adds', 'measures', 'supports',
    'catalyzes', 'runs', 'sends', 'manages', 'divides', 'builds', 'grows', 'contains'
]
VERBS += [
    'denotes', 'means', 'implies', 'signifies', 'describes',
    'indicates', 'entails', 'represents',
    'eat', 'eats',            # → “bears, eat, …”
    'glows', 'glow', 'sheds', 'decomposes', 'reacts',
    'allows', 'transmits', 'emits', 'returns', 'benefits', 'spreads',
    'lives', 'thrives', 'walks', 'runs', 'converts', 'prevents',
    'withstands', 'orbits', 'dissolves', 'repels'
]
VERB_RX = re.compile(r'\s(' + '|'.join(VERBS) + r')\s', flags=re.I)



def split_phrases_arg_ins(sentence: str) -> list[str]:
    """Return a list of comma-separated units for one sentence."""
    s = ' ' + sentence.strip() + ' '
    for old, new in REPLACEMENTS:
        s = s.replace(old, new)
    s = PREP_FALLBACK.sub(r', \1, \2', s)
    return [p.strip() for p in s.split(',') if p.strip()]


def split_phrases_arg_pred(sentence: str) -> list[str]:
    """Return a list of comma-delimited units for one sentence."""
    s = ' ' + sentence.strip() + ' '

    # --- step 1: fixed, multi-word phrases ----------------------------------
    for old, new in LONG_PHRASE_REPL:
        s = s.replace(old, new)

    # --- step 2: shorter patterns handled by regex --------------------------
    s = IS_A_RX.sub(', is a, ', s)
    s = IS_RX.sub(', is, ', s)
    s = HAS_RX.sub(', has, ', s)
    s = VERB_RX.sub(lambda m: f', {m.group(1)}, ', s)

    # --- final clean-up ------------------------------------------------------
    parts = [p.strip() for p in s.split(',') if p.strip()]
    return parts


def widen_csv(src: str, dst: str) -> None:
    df = pd.read_csv(src)
    rows = []

    for _, row in df.iterrows():
        pieces = []
        for col in df.columns:
            pieces.extend(split_phrases_arg_pred(row[col]))
        rows.append(pieces)

    max_len = max(len(r) for r in rows)
    rows = [r + ['']*(max_len-len(r)) for r in rows]
    out = pd.DataFrame(rows, columns=[f'C{i}' for i in range(1, max_len+1)])

    out.to_csv(
        dst,
        index=False,
        quoting=csv.QUOTE_NONE,   # keep it quote-free
        escapechar='\\',
    )
    print(f'✔ Wrote {dst}   [{len(out)} rows × {len(out.columns)} columns]')


widen_csv("/home/oscb/Mechanistic-Interpretation-Material-Inference/mimi/data/examples_100/arg_substitution_100.csv", "arg_sub.csv")

✔ Wrote arg_sub.csv   [100 rows × 11 columns]
